## MLflow's Model Registry 
(Unnecessary to rerun it cause the versions were choosen manually looking at mlflow so if it is run again new versions will be created pointlesly)
To run this you need to launch the mlflow server in Fargate with the corresponding Github action.

In [1]:
Fargate_tracking_uri = "http://mlflo-mlflo-1t73jy0dxw3bw-f2c1f1638afa4ab6.elb.eu-central-1.amazonaws.com/"

In [2]:
import mlflow
mlflow.set_tracking_uri(Fargate_tracking_uri)#NOTE: Important!!!  tput the "http://" at the beginning or it will not work properly
#NOTE: Important!!!  to set the tracking uri here cause otherwise it stores the artifact locally. 
#NOTE: Important!!! The uri wil change every time you start the tracking server on Fargate so you have to change it every time
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://mlflo-mlflo-1t73jy0dxw3bw-f2c1f1638afa4ab6.elb.eu-central-1.amazonaws.com/'


In [4]:
from  mlflow.tracking import MlflowClient
client = MlflowClient()
experiments = client.search_experiments() # returns a list of mlflow.entities.Experiment
experiments

[<Experiment: artifact_location='s3://34aghxe-mlflow-server-artifacts-141507290110/3', creation_time=1670176563970, experiment_id='3', last_update_time=1670176563970, lifecycle_stage='active', name='experiment-covid-2', tags={}>,
 <Experiment: artifact_location='s3://34aghxe-mlflow-server-artifacts-141507290110/2', creation_time=1670176453890, experiment_id='2', last_update_time=1670176453890, lifecycle_stage='active', name='experiment-covid-3', tags={}>,
 <Experiment: artifact_location='s3://34aghxe-mlflow-server-artifacts-141507290110/1', creation_time=1670176225608, experiment_id='1', last_update_time=1670176225608, lifecycle_stage='active', name='my-experimenta', tags={}>,
 <Experiment: artifact_location='s3://34aghxe-mlflow-server-artifacts-141507290110/0', creation_time=1670175501589, experiment_id='0', last_update_time=1670175501589, lifecycle_stage='active', name='Default', tags={}>]

Let's check the latest versions for the experiment with id `2`...

In [5]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='2',
    # name='experiment-covid-1',
    # filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metric.evaluated_RMSE"]
)

In [6]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['evaluated_RMSE']:.4f}")

run id: 0c2df617d52d47a08f3f840daffc19a9, rmse: 0.1706


Comparing the experiments in mlflow the first one has less evaluated RSME so I will choose that model => RUNID = 0c2df617d52d47a08f3f840daffc19a9

### Interacting with the Model Registry

In this section I will use the `MlflowClient` instance to:

1. Register a new version for the experiment `experiment-covid-2`
2. Retrieve the latests versions of the model `covid-predictor` and check that a new version was created.


In [5]:
#NOTE: this is commented because only necessary if not done previous task of the notebook before
# import mlflow
# mlflow.set_tracking_uri(Fargate_tracking_uri)

In [7]:
run_id = "0c2df617d52d47a08f3f840daffc19a9" #This is not the run_id that will be chosen but it is to test that we can set it on stage "Archived"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="covid-predictor") #this is to register/create a new version of the model

Successfully registered model 'covid-predictor'.
2022/12/04 19:07:46 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: covid-predictor, version 1
Created version '1' of model 'covid-predictor'.


<ModelVersion: creation_timestamp=1670177265606, current_stage='None', description='', last_updated_timestamp=1670177265606, name='covid-predictor', run_id='0c2df617d52d47a08f3f840daffc19a9', run_link='', source='s3://34aghxe-mlflow-server-artifacts-141507290110/2/0c2df617d52d47a08f3f840daffc19a9/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [8]:
model_uri

'runs:/0c2df617d52d47a08f3f840daffc19a9/model'

In [9]:
model_name = "covid-predictor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


In [10]:
model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1670177265606, current_stage='Staging', description='', last_updated_timestamp=1670177352779, name='covid-predictor', run_id='0c2df617d52d47a08f3f840daffc19a9', run_link='', source='s3://34aghxe-mlflow-server-artifacts-141507290110/2/0c2df617d52d47a08f3f840daffc19a9/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [11]:
model_name = "covid-predictor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging


#### Now we register the model that we want to use and set it in stage Production

In [12]:
run_id = "6509bec6c96d4f9d8e1b88c0812e1590" #This is not the run_id that will be chosen but it is to test that we can set it on stage "Archived"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="covid-predictor") #this is to register/create a new version of the model

Registered model 'covid-predictor' already exists. Creating a new version of this model...
2022/12/04 19:10:43 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: covid-predictor, version 2
Created version '2' of model 'covid-predictor'.


<ModelVersion: creation_timestamp=1670177442560, current_stage='None', description='', last_updated_timestamp=1670177442560, name='covid-predictor', run_id='6509bec6c96d4f9d8e1b88c0812e1590', run_link='', source='s3://34aghxe-mlflow-server-artifacts-141507290110/3/6509bec6c96d4f9d8e1b88c0812e1590/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>

In [13]:
model_name = "covid-predictor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 2, stage: None


In [14]:
model_version = 2
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1670177442560, current_stage='Production', description='', last_updated_timestamp=1670177457610, name='covid-predictor', run_id='6509bec6c96d4f9d8e1b88c0812e1590', run_link='', source='s3://34aghxe-mlflow-server-artifacts-141507290110/3/6509bec6c96d4f9d8e1b88c0812e1590/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>

In [15]:
model_name = "covid-predictor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 2, stage: Production


In [16]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"Jrv_note: The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1670177442560, current_stage='Production', description='Jrv_note: The model version 2 was transitioned to Production on 2022-12-04', last_updated_timestamp=1670177481721, name='covid-predictor', run_id='6509bec6c96d4f9d8e1b88c0812e1590', run_link='', source='s3://34aghxe-mlflow-server-artifacts-141507290110/3/6509bec6c96d4f9d8e1b88c0812e1590/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Download the model 
2. Test that the model works, make predictions and also check the rmse is low Load the test dataset, which corresponds to the last 7 days.
3. Download the model that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.

**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [17]:
#Download model 
run_id = "6509bec6c96d4f9d8e1b88c0812e1590" 
client.download_artifacts(run_id=run_id, path='models', dst_path='.')

C:\Users\jraldua-veuthey\AppData\Local\Temp\ipykernel_4804\883153170.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  client.download_artifacts(run_id=run_id, path='models', dst_path='.')


'c:\\Users\\jraldua-veuthey\\Documents\\Github_NoPycharm\\mlops-covid\\exp-track-mod-reg-mlflowFargate\\models'

In [18]:
import pickle

with open("models/model.pkl", "rb") as f_in:
    model = pickle.load(f_in)

In [19]:
model

Pipeline(steps=[('poly', PolynomialFeatures(degree=3, include_bias=False)),
                ('linear', LinearRegression())])

In [20]:
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
from pandas._libs.tslibs.timestamps import Timestamp

TARGETS = ["ConfirmedCases", "Fatalities"]
features = ["prev_{}".format(col) for col in TARGETS]
loc_group = ["Province_State", "Country_Region"]

def preprocess(df):
    df["Date"] = df["Date"].astype("datetime64[ms]")
    for col in loc_group:
        df[col].fillna("none", inplace=True) #NOTE: replace all NaN with none  
    for col in TARGETS:
        df[col] = np.log1p(df[col]) 
    for col in TARGETS:
        df["prev_{}".format(col)] = df.groupby(loc_group)[col].shift() #NOTE: the prev_ columns basically has the same than the others but delayed one day
    return df

def get_data_last_days(num_days): #gets the data from the last "num_days" days
    num_days = num_days + 2 #I do this because I get rid of the first date since it has NaNs in the columns prev_ConfirmedCases	prev_Fatalities and because of the for loop with range
    dfs = []  # empty list which will hold your dataframes
    for d in range(1, num_days): #NOTE: do the same that has been done for the first day but for the whole period
        date = datetime.now() - timedelta(days=d)
        date_str = date.strftime("%m-%d-%Y")
        source_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/' + date_str + '.csv'
        df_temp = pd.read_csv(source_url)
        df_temp.rename(columns={"Last_Update": "Date"}, inplace=True) #Renane dataframe column from "Last_Update" to "Date"
        df_temp_2 = df_temp[["Admin2", "Province_State", "Country_Region","Confirmed", "Deaths"]].copy() #TODO: consider also other columns in future versions like Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
        df_temp_2.loc[:,"Date"] = date.strftime("%Y-%m-%d") 
        dfs.append(df_temp_2)  # append dataframe to list
    res = pd.concat(dfs, ignore_index=True)  # concatenate list of dataframes
    
    # group by Country_Region and sum Confirmed and Deaths
    df = res.groupby(['Province_State','Country_Region','Date']).agg({'Confirmed':'sum', 'Deaths':'sum'})
    df.reset_index(inplace=True)
    df.rename(columns={"Confirmed": "ConfirmedCases", "Deaths": "Fatalities"}, inplace=True)
        
    df = preprocess(df)
    
    df = df[df["Date"] > df["Date"].min()].copy() #removes the first day since it has NaNs in the "prev" columns

    df.reset_index(inplace=True, drop=True)
    
    return df

def predict_today_Province_State(model,Province_State):
    df = get_data_last_days(1) #Get data from yesterday
    y_pred = predict_today_world(model) #Predict today worldwide
    index_PS = df[df['Province_State']==Province_State].iloc[0].name
    predictions = y_pred[index_PS]
    return predictions #First the predicted Confirmed cases and second the predicted fatalities

def predict_today_world(model):#Does the prediction for today
    df = get_data_last_days(1) #Get data from yesterday
    yesterday = datetime.now() - timedelta(days=1)
    yesterday = yesterday.replace(hour=0, minute=0, second=0, microsecond=0)
    yesterday = Timestamp(yesterday)
    y_pred = np.clip(model.predict(df.loc[df["Date"] == yesterday][features]), None, 16)#NOTE: here predicting the targets for the first day and saturating (clip) them with max=16
    return y_pred


def evaluate_yesterday():
    return evaluate_last_days(1)

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def predict_past(model, num_days):
    test_df = get_data_last_days(num_days)
    first_day = datetime.now() - timedelta(days=num_days)
    first_day = first_day.replace(hour=0, minute=0, second=0, microsecond=0)
    first_day = Timestamp(first_day)
    y_pred = np.clip(model.predict(test_df.loc[test_df["Date"] == first_day][features]), None, 16)#NOTE: here he is predicting the targets for the first day and saturating (clip) them with max=16
 
    for i, col in enumerate(TARGETS):
        test_df["pred_{}".format(col)] = 0
        test_df.loc[test_df["Date"] == first_day, "pred_{}".format(col)] = y_pred[:, i] #NOTE: here he sets the predicted columns

    for d in range(1, num_days): #NOTE: do the same that has been done for the first day but for the whole period
        y_pred = np.clip(model.predict(y_pred), None, 16)
        date = first_day + timedelta(days=d)

    for i, col in enumerate(TARGETS):
        test_df.loc[test_df["Date"] == date, "pred_{}".format(col)] = y_pred[:, i]

    return test_df

def evaluate_last_days(model,num_days):
    #get data from the last "num_days" days
    df = predict_past(model,num_days)
    
    #get the rmse
    error = 0
    for col in TARGETS:
        error += rmse(df[col].values, df["pred_{}".format(col)].values) #NOTE: checks the error between the predicted columns and the target columns
    return np.round(error/len(TARGETS), 5)


In [21]:
evaluate_last_days(model,5)

c:\Users\jraldua-veuthey\.virtualenvs\mlops-covid-0dX7y8J5\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
c:\Users\jraldua-veuthey\.virtualenvs\mlops-covid-0dX7y8J5\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
c:\Users\jraldua-veuthey\.virtualenvs\mlops-covid-0dX7y8J5\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
c:\Users\jraldua-veuthey\.virtualenvs\mlops-covid-0dX7y8J5\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(


7.6173

In [22]:
predict_today_Province_State(model,'Madrid')

array([14.49384131,  9.90338004])

In [23]:
ans = predict_today_Province_State(model,'Madrid')
df_ans = pd.DataFrame()
df_ans['pred_ConfirmedCases']=pd.Series(ans[0])
df_ans['pred_Fatalities']=pd.Series(ans[1])
df_ans

,pred_ConfirmedCases,pred_Fatalities
0,14.493841,9.90338


In [24]:
model_name

'covid-predictor'

Based on the results, the model that is currently on "Production" works fine (has low RMSE) so no need to change stage